In [1]:
import gzip

from gensim.models import Word2Vec, LdaMulticore
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary


import numpy as np
import pandas as pd

import pyLDAvis
from pyLDAvis import gensim
 
from sklearn.decomposition import PCA

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
ls data

reviews_data.txt  reviews_data.txt.gz


In [3]:
def preprocessing(sentence):
    return [word for word in simple_preprocess(sentence) \
             if word not in STOPWORDS]
    
def read_sentences(filename):
    with gzip.open(filename, 'rb') as f:
        for line in f:
            yield preprocessing(line)
        
%time sentences = list(read_sentences('data/reviews_data.txt.gz'))

CPU times: user 40.4 s, sys: 530 ms, total: 40.9 s
Wall time: 40.9 s


In [5]:
model = Word2Vec.load('models/hotel_model_size100_window10_mincount2.model')

In [25]:
%time model = Word2Vec(sentences, size=100, window=5, min_count=2)

CPU times: user 3min 16s, sys: 565 ms, total: 3min 16s
Wall time: 1min 9s


# Reprezentacja wektorowa

In [6]:
model.wv.most_similar('bad')

[('terrible', 0.7091025114059448),
 ('horrible', 0.6725283861160278),
 ('awful', 0.6613410115242004),
 ('okay', 0.6073172092437744),
 ('sucked', 0.6060259342193604),
 ('poor', 0.6045093536376953),
 ('crappy', 0.6011784076690674),
 ('ok', 0.5935438871383667),
 ('complain', 0.5927609205245972),
 ('worse', 0.5925099849700928)]

In [7]:
model.wv.most_similar(positive=['bad', 'terrible'], negative=['good'])

[('horrible', 0.8423725962638855),
 ('awful', 0.8038561940193176),
 ('horrific', 0.7360600233078003),
 ('horrid', 0.7299700379371643),
 ('dreadful', 0.7213677167892456),
 ('shocking', 0.6904001235961914),
 ('nasty', 0.6877930760383606),
 ('disaster', 0.6822035908699036),
 ('appalling', 0.6815707683563232),
 ('horrendous', 0.6786438822746277)]

In [35]:
model.wv.most_similar('london')

[('nyc', 0.8440564870834351),
 ('manhattan', 0.806428074836731),
 ('sf', 0.8048614263534546),
 ('ny', 0.796454131603241),
 ('chicago', 0.7588106393814087),
 ('beijing', 0.7530739903450012),
 ('dubai', 0.7501722574234009),
 ('montreal', 0.7497483491897583),
 ('manhatten', 0.7044616937637329),
 ('lv', 0.6872093677520752)]

# Modelowanie tematow - LDA

In [8]:
%time sentences_light = np.random.permutation(sentences)

CPU times: user 2.13 s, sys: 112 ms, total: 2.24 s
Wall time: 2.23 s


In [9]:
sentences_light = sentences_light[:1000]
dictionary = Dictionary(sentences_light)

In [10]:
#list(dictionary.items())

In [11]:
dictionary.doc2bow(['car', 'car', 'plane'])

[(148, 2), (4100, 1)]

In [12]:
dictionary.token2id['car']

148

In [13]:
bow_corpus = [dictionary.doc2bow(sent) for sent in sentences_light]

In [14]:
%time lda_model = LdaMulticore(bow_corpus, id2word=dictionary, num_topics=100, passes=20, workers=8)

CPU times: user 5min 7s, sys: 4min 53s, total: 10min
Wall time: 1min 29s


In [17]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.021*"best" + 0.016*"saved" + 0.015*"street" + 0.015*"trip" + 0.015*"place" + 0.014*"nice" + 0.014*"great" + 0.012*"stay" + 0.010*"outside" + 0.010*"excellent"
Topic: 1 
Words: 0.022*"hotel" + 0.012*"time" + 0.011*"stay" + 0.010*"like" + 0.009*"rooms" + 0.009*"quot" + 0.009*"timeshare" + 0.009*"pretty" + 0.008*"saturday" + 0.008*"nice"
Topic: 2 
Words: 0.026*"room" + 0.012*"fi" + 0.010*"london" + 0.009*"small" + 0.009*"wetherspoons" + 0.009*"im" + 0.009*"midnight" + 0.009*"wi" + 0.008*"point" + 0.008*"clean"
Topic: 3 
Words: 0.028*"room" + 0.026*"hotel" + 0.015*"location" + 0.012*"clean" + 0.011*"great" + 0.011*"quot" + 0.011*"stayed" + 0.010*"nights" + 0.009*"breakfast" + 0.008*"staff"
Topic: 4 
Words: 0.023*"room" + 0.018*"great" + 0.014*"hotel" + 0.013*"like" + 0.012*"stay" + 0.010*"new" + 0.009*"amp" + 0.009*"york" + 0.009*"hotels" + 0.009*"free"
Topic: 5 
Words: 0.027*"hotel" + 0.014*"comfortable" + 0.013*"room" + 0.013*"bed" + 0.009*"great" + 0.009*"nicely" + 0.

In [18]:
lda_vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

/home/lkoziol/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [20]:
pyLDAvis.display(lda_vis)